<a href="https://colab.research.google.com/github/naralakamsani/supervised_learning/blob/main/reservation_retention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports

In [18]:
from google.colab import drive

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.under_sampling import RandomUnderSampler

from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

#Data

[Hotel Reservation](https://www.kaggle.com/datasets/ahsan81/hotel-reservations-classification-dataset?select=Hotel+Reservations.csv)


**Load Data**

In [19]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
df = pd.read_csv("/content/drive/MyDrive/CS7641/Data/Hotel_Reservations/Hotel Reservations.csv")

In [21]:
df.head()

,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,INN00001,2,0,1,2,Meal Plan 1,0,Room_Type 1,224,2017,10,2,Offline,0,0,0,65.00,0,Not_Canceled
1,INN00002,2,0,2,3,Not Selected,0,Room_Type 1,5,2018,11,6,Online,0,0,0,106.68,1,Not_Canceled
2,INN00003,1,0,2,1,Meal Plan 1,0,Room_Type 1,1,2018,2,28,Online,0,0,0,60.00,0,Canceled
3,INN00004,2,0,0,2,Meal Plan 1,0,Room_Type 1,211,2018,5,20,Online,0,0,0,100.00,0,Canceled
4,INN00005,2,0,1,1,Not Selected,0,Room_Type 1,48,2018,4,11,Online,0,0,0,94.50,0,Canceled


**Encode Labeled Data**

In [22]:
df = df.drop(columns=["Booking_ID"])

df['booking_status'].replace('Not_Canceled', 0, inplace=True)
df['booking_status'].replace('Canceled', 1, inplace=True)

df['type_of_meal_plan'].replace('Not Selected', 0, inplace=True)
df['type_of_meal_plan'].replace('Meal Plan 1', 1, inplace=True)
df['type_of_meal_plan'].replace('Meal Plan 2', 2, inplace=True)
df['type_of_meal_plan'].replace('Meal Plan 3', 3, inplace=True)

df['room_type_reserved'].replace('Room_Type 1', 1, inplace=True)
df['room_type_reserved'].replace('Room_Type 2', 2, inplace=True)
df['room_type_reserved'].replace('Room_Type 3', 3, inplace=True)
df['room_type_reserved'].replace('Room_Type 4', 4, inplace=True)
df['room_type_reserved'].replace('Room_Type 5', 5, inplace=True)
df['room_type_reserved'].replace('Room_Type 6', 6, inplace=True)
df['room_type_reserved'].replace('Room_Type 7', 7, inplace=True)

df['market_segment_type'].replace('Offline', 0, inplace=True)
df['market_segment_type'].replace('Online', 1, inplace=True)
df['market_segment_type'].replace('Corporate', 2, inplace=True)
df['market_segment_type'].replace('Aviation', 3, inplace=True)
df['market_segment_type'].replace('Complementary', 4, inplace=True)

In [23]:
df.head()

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,2,0,1,2,1,0,1,224,2017,10,2,0,0,0,0,65.00,0,0
1,2,0,2,3,0,0,1,5,2018,11,6,1,0,0,0,106.68,1,0
2,1,0,2,1,1,0,1,1,2018,2,28,1,0,0,0,60.00,0,1
3,2,0,0,2,1,0,1,211,2018,5,20,1,0,0,0,100.00,0,1
4,2,0,1,1,0,0,1,48,2018,4,11,1,0,0,0,94.50,0,1


**Split Data**

In [24]:
x_train, x_test, y_train, y_test = train_test_split(df.drop(['booking_status'],axis=1), df['booking_status'], test_size=0.2, random_state=1)

In [25]:
print("Train data size:", len(x_train))
print("Test data size:", len(x_test))

Train data size: 29020
Test data size: 7255


**Balance Dataset**

In [26]:
# rus = RandomUnderSampler(random_state=1)
# x_train, y_train = rus.fit_resample(x_train, y_train)

In [27]:
print('Training Set Class Balance:')
print(y_train.value_counts())

Training Set Class Balance:
0    19483
1     9537
Name: booking_status, dtype: int64


**Scale Data**

In [28]:
# scaler = StandardScaler()
# x_train = pd.DataFrame(scaler.fit_transform(x_train), columns=x_train.columns)
# x_test = pd.DataFrame(scaler.fit_transform(x_test), columns=x_train.columns)

In [29]:
x_train.head()

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests
33501,1,0,2,3,1,0,1,115,2018,2,27,0,0,0,0,76.0,0
6136,2,0,2,1,2,0,1,131,2017,8,15,0,0,0,0,94.5,0
28204,2,0,1,0,1,0,4,16,2018,9,26,1,0,0,0,164.0,0
32311,2,0,2,2,2,0,1,169,2018,4,22,0,0,0,0,106.0,0
8622,1,0,1,0,1,1,1,5,2018,12,5,4,1,2,8,0.0,1


# Decision Tree

In [30]:
dt_model = DecisionTreeClassifier(random_state=1)
dt_model.fit(x_train, y_train)

DecisionTreeClassifier(random_state=1)

In [31]:
y_pred = dt_model.predict(x_test)
print("Classification Report:\n", classification_report(y_test, y_pred))

Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.91      0.91      4907
           1       0.81      0.81      0.81      2348

    accuracy                           0.88      7255
   macro avg       0.86      0.86      0.86      7255
weighted avg       0.88      0.88      0.88      7255



#Neural Network


In [32]:
nn_model = MLPClassifier(random_state=1, hidden_layer_sizes=(100,100))
nn_model.fit(x_train, y_train)

MLPClassifier(hidden_layer_sizes=(100, 100), random_state=1)

In [33]:
y_pred = nn_model.predict(x_test)
print("Classification Report:\n", classification_report(y_test, y_pred))

Classification Report:
               precision    recall  f1-score   support

           0       0.78      0.96      0.86      4907
           1       0.84      0.43      0.56      2348

    accuracy                           0.79      7255
   macro avg       0.81      0.69      0.71      7255
weighted avg       0.80      0.79      0.76      7255



#Boosting


In [34]:
boosting_model = AdaBoostClassifier(random_state=1)
boosting_model.fit(x_train, y_train)

AdaBoostClassifier(random_state=1)

In [35]:
y_pred = boosting_model.predict(x_test)
print("Classification Report:\n", classification_report(y_test, y_pred))

Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.89      0.87      4907
           1       0.74      0.67      0.70      2348

    accuracy                           0.82      7255
   macro avg       0.79      0.78      0.78      7255
weighted avg       0.81      0.82      0.81      7255



#Support Vector Machine

In [36]:
sv_model = SVC()
sv_model.fit(x_train, y_train)

SVC()

In [37]:
y_pred = sv_model.predict(x_test)
print("Classification Report:\n", classification_report(y_test, y_pred))

Classification Report:
               precision    recall  f1-score   support

           0       0.77      0.93      0.84      4907
           1       0.75      0.41      0.53      2348

    accuracy                           0.77      7255
   macro avg       0.76      0.67      0.69      7255
weighted avg       0.76      0.77      0.74      7255



#k-Nearest Neighbor

In [38]:
knn_model = KNeighborsClassifier()
knn_model.fit(x_train, y_train)

KNeighborsClassifier()

In [39]:
y_pred = knn_model.predict(x_test)
print("Classification Report:\n", classification_report(y_test, y_pred))

Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.89      0.86      4907
           1       0.74      0.63      0.68      2348

    accuracy                           0.81      7255
   macro avg       0.79      0.76      0.77      7255
weighted avg       0.80      0.81      0.80      7255

